### **Делатель двух маленьких справочников и справочника для поиска населённых пунктов**   

**Архивирование:**   
`zip_path = shutil.make_archive(dir_name, 'zip', dir_path.replace("\\", "/") + "/" + dir_name)`     
`dir_name` - имя архива, просто без адреса: 'справочники_2023-07-11' (тогда сохранит в папку скрипта!)        
`dir_path.replace("\\", "/") + "/" + dir_name` - путь к папке, где лежат файлы для архивации: 'c:/work/scripts/справочники/справочники_2023-07-11'     
`zip_path = zip_path.replace("\\", "/")` - путь к архиву: 'c:/work/scripts/справочники/справочники_2023-07-11.zip'      
**Удаление папки:**     
`shutil.rmtree(dir_path.replace("\\", "/") + "/" + dir_name)`     

## Один общий скрипт

In [1]:
%%time

import pandas as pd
from datetime import date
import os
import sys
import shutil # упаковка в архив
import warnings # игнорирование предупреждения о формате при импорте из excel

warnings.simplefilter("ignore")

# пути к файлам и версии справочников (из названия файлов)
dir_path = os.getcwd()
dir_ = os.listdir(dir_path.replace("\\", "/"))
for file in dir_:
    if file.split("_")[0].split(".")[-1] == "1461":
        path_mo = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            mo_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
    if file.split("_")[0].split(".")[-1] == "114":
        path_dept = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            sp_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
    if file.split("_")[0].split(".")[-1] == "115":
        path_room = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            room_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
# path_save_sp = dir_path.replace("\\", "/") + "/sp.csv"
# path_save_room = dir_path.replace("\\", "/") + "/room.csv"
print("Версии справочников, извлечённые из названий файлов:")
print(
    f"\t- справочник МО: {mo_version}\n\t- справочник СП: {sp_version}\n\t- справочник кабинетов: {room_version}"
)
if path_mo.split('.')[-1] == 'zip' or path_dept.split('.')[-2] == 'zip' or path_room.split('.')[-2] == 'zip':
    print('Скачанные архивы не распакованы, программа завершает работу.\nПопробуйте распаковать архивы.')
    sys.exit()

print('Загрузка справочников:')

df_mo = pd.read_excel(
    path_mo, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник МО")
df_dept = pd.read_excel(
    path_dept, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник СП")
df_room = pd.read_excel(
    path_room, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник кабинетов")


print("Формирование маленьких справочников")

# создание папки для сохранения маленьких справочников
dir_name = "справочники_" + str(date.today())
try:
    os.mkdir(dir_name)
except Exception:
    pass

# пути сохранения маленьких справочников
path_save_dept = dir_path.replace("\\", "/") + "/" + dir_name + "/dept.csv"
path_save_room = dir_path.replace("\\", "/") + "/" + dir_name + "/room.csv"
path_save_mo = dir_path.replace("\\", "/") + "/" + dir_name + "/mo_short.csv"

# выбор полей для маленьких справочников
col_mo = [
    "oid",
    "nameFull",
    "nameShort",
    "regionName",
    "moDeptName",
]
col_room = [
    "mo_oid",
    "depart_oid",
    "depart_name",
    "depart_type_name",
    "depart_kind_name",
    "oid",
    "ambulance_subdivision_name",
    "ambulance_room_count",
    "building_address_region_name",
]
col_dept = [
    "mo_oid",
    "depart_oid",
    "depart_name",
    "depart_type_name",
    "depart_kind_name",
    "separate_depart_boolean",
    "building_address_region_name",
    "hospital_home_building_address_region_name",
]

df_mo_ref = df_mo[[*col_mo]].rename(columns={"oid": "mo_oid"})
df_dept_ref = df_dept[[*col_dept]]
df_room_ref = df_room[[*col_room]].rename(columns={"oid": "room_oid"})
df_room_ref["ambulance_room_count"] = df_room_ref["ambulance_room_count"].fillna(0)

df_res_dept = df_dept_ref.merge(df_mo_ref, left_on = "mo_oid", right_on = "mo_oid", how = "left").reset_index(drop = True)
df_res_room = df_room_ref.merge(df_mo_ref, left_on = "mo_oid", right_on = "mo_oid", how = "left").reset_index(drop = True)
print("Маленькие справочники сформированы")


# сохранение
df_res_dept.to_csv(path_save_dept, sep=";", encoding="utf-8-sig", index=False)
df_res_room.to_csv(path_save_room, sep=";", encoding="utf-8-sig", index=False)
df_mo.to_csv(path_save_mo, sep=";", encoding="utf-8-sig", index=False)

# упаковка в архив
zip_path = shutil.make_archive(dir_name, 'zip', dir_path.replace("\\", "/") + "/" + dir_name)
zip_path = zip_path.replace("\\", "/")

# удаление справочников из неархивированной папки и её самой
shutil.rmtree(dir_path.replace("\\", "/") + "/" + dir_name)

print(f"Маленькие справочники упакованы в архив: {zip_path}")
print("Формирование справочников для поиска населённых пунктов")

# создание папки для сохранения результатов
dir_name = "для_поиска_НП_" + str(date.today())
try:
    os.mkdir(dir_name)
except Exception:
    pass

# выбор полей для справочника НП
col_mo = [
    "oid",
    "nameFull",
    "nameShort",
    "regionName",
    "moDeptName",
    "areaName",
    "prefixArea",
]
col_room = [
    "mo_oid",
    "depart_oid",
    "depart_name",
    "depart_type_name",
    "depart_kind_name",
    "oid",
    "ambulance_subdivision_name",
    "ambulance_room_count",
    "building_address_prefix_area",
    "building_address_area_name",
    "building_address_region_name",
]
col_dept = [
    "mo_oid",
    "depart_oid",
    "depart_name",
    "depart_type_name",
    "depart_kind_name",
    "separate_depart_boolean",
    "building_address_prefix_area",
    "building_address_area_name",
    "building_address_region_name",
    "hospital_home_building_address_region_name",
]

df_mo_loc = df_mo[[*col_mo]].rename(columns={"oid": "mo_oid"})
df_mo_loc = df_mo_loc.fillna("")
df_mo_loc = df_mo_loc.drop(index=0).reset_index(drop=True)

df_dept_loc = df_dept[[*col_dept]]
df_dept_loc = df_dept_loc.fillna("")
df_dept_loc = df_dept_loc.drop(index=0).reset_index(drop=True)

df_room_loc = df_room[[*col_room]].rename(columns={"oid": "room_oid"})
df_room_loc = df_room_loc.rename(columns={"oid": "room_oid"})
df_room_loc["ambulance_room_count"] = df_room_loc["ambulance_room_count"].fillna(0)
df_room_loc = df_room_loc.drop(index=0).reset_index(drop=True)

# формирование правильного справочника МО
df_mo_loc["locality_name"] = df_mo_loc.apply(
    lambda row: str(row["prefixArea"]) + "_" + str(row["areaName"])
    if row["areaName"] != ""
    else "",
    axis=1,
)
print('\t- справочник МО для поиска НП сформирован')

# формирование правильного справочника СП
df_dept_loc["locality_name"] = df_dept_loc.apply(
    lambda row: str(row["building_address_prefix_area"])
    + "_"
    + str(row["building_address_area_name"])
    if row["building_address_area_name"] != ""
    else "",
    axis=1,
)
print('\t- справочник СП для поиска НП сформирован')

# формирование правильного справочника кабинетов
df_room_loc["ambulance_room_count"] = df_room_loc["ambulance_room_count"].astype("int")
df_room_loc = df_room_loc.fillna("")
df_room_loc["locality_name"] = df_room_loc.apply(
    lambda row: str(row["building_address_prefix_area"])
    + "_"
    + str(row["building_address_area_name"])
    if row["building_address_area_name"] != ""
    else "",
    axis=1,
)
print('\t- справочник кабинетов для поиска НП сформирован')

# разъяснения для юзера
df_notation = pd.DataFrame(
    {
        "Столбец": ["F", "D", "I", "K", "Y", "Z", "AA", "AB", "AC", "AD", "AE"],
        "Имя столбца": [
            "OID СП",
            "OID МО",
            "Название типа структурного подразделения или название типа медицинской организации",
            "Количество кабинетов из ФРМО",
            "населённый пункт oid_МО",
            "населённый пункт oid_СП",
            "населённый пункт из oid_каб как СП",
            "населённый пункт из oid_каб как каб",
            "ПБ_ПНД?",
            "Населённый пункт из логики",
            "Населённый пункт итог",
        ],
        "Формула": [
            "",
            "",
            "",
            "*=ЕСЛИОШИБКА(ВПР($F2;каб!$F$3:$L$700000;3;0);1)",
            '*=ЕСЛИОШИБКА(ВПР(D2;МО!A:H;8;0);"")',
            '*=ЕСЛИОШИБКА(ВПР(F2;СП!B:K;10;0);"")',
            '*=ЕСЛИОШИБКА(ВПР(F2;каб!B:L;11;0);"")',
            '*=ЕСЛИОШИБКА(ВПР(F2;каб!F:L;7;0);"")',
            '*=ЕСЛИ(ИЛИ(I2="Психоневрологический диспансер (центр психического здоровья, диспансерное отделение психиатрической больницы)";I2="Психиатрическая больница");1;"")',
            '*=ЕСЛИОШИБКА(ЕСЛИ(ИЛИ(AC2=1;F2="");ВПР(D2;МО!A:H;8;0);ЕСЛИОШИБКА(ВПР(F2;каб!F:L;7;0); ЕСЛИОШИБКА(ВПР(F2;каб!B:L;11;0);ВПР(F2;СП!B:K;10;0))));Y2)',
            '*=ЕСЛИ(И(Y2="";AD2="");ИНДЕКС(Z2:AB2;1;ПОИСКПОЗ(1;ИНДЕКС(1-ЕПУСТО(Z2:AB2);1;0);0));ЕСЛИ(ИЛИ(AD2="";AD2="_";AD2=0);Y2;AD2))',
        ],
    }
)

# создание Pandas Excel writer, используя XlsxWriter в качестве движка
path_save = dir_path.replace("\\", "/") + "/" + dir_name + "/ref_locality.xlsx"
writer = pd.ExcelWriter(path_save, engine="xlsxwriter")

# write each DataFrame to a specific sheet
df_mo_loc.to_excel(writer, sheet_name="МО", index=False)
df_dept_loc.to_excel(writer, sheet_name="СП", index=False)
df_room_loc.to_excel(writer, sheet_name="каб", index=False)
df_notation.to_excel(writer, sheet_name="формулы", index=False)

# close the Pandas Excel writer and output the Excel file
writer.close()

# архивирование
zip_path = shutil.make_archive(
    dir_path.replace("\\", "/") + "/" + dir_name,
    "zip",
    dir_path.replace("\\", "/") + "/" + dir_name,
)

# удаление папки
shutil.rmtree(dir_path.replace("\\", "/") + "/" + dir_name)

print(f"Справочники для поиска НП упакованы в архив: {zip_path}")
print('ВСЁ!')

Версии справочников, извлечённые из названий файлов:
	- справочник МО: 6.1138
	- справочник СП: 2.1505
	- справочник кабинетов: 2.1449
Загрузка справочников:
	- загружен справочник МО
	- загружен справочник СП
	- загружен справочник кабинетов
Формирование маленьких справочников
Маленькие справочники сформированы
Маленькие справочники упакованы в архив: c:/work/scripts/справочники/справочники_2023-11-07.zip
Формирование справочников для поиска населённых пунктов
	- справочник МО для поиска НП сформирован
	- справочник СП для поиска НП сформирован
	- справочник кабинетов для поиска НП сформирован
Справочники для поиска НП упакованы в архив: c:\work\scripts\справочники\для_поиска_НП_2023-11-07.zip
ВСЁ!
CPU times: total: 3min 48s
Wall time: 12min 44s
